In [6]:
import pandas as pd
import re
def unicode(match):
    match = match.group()
    return chr(int(match[2:], base=16))
list_csv = ['data/kor/attraction_review',
            'data/kor/hotel_review',
            'data/kor/restaurant_review',
            'data/eng/eng_attraction_review',
            'data/eng/eng_hotel_review',
            'data/eng/eng_restaurant_review']

In [7]:
print('In \'userId\' column')
for csv in list_csv:
    df = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='euc-kr')
    print('na',len(df[df['userId'].isna()]),'개 ',end='')
    count = 0
    for i in df['userId']:
        if not pd.isna(i) and (re.fullmatch(r'^ *$', i) != None):
            count+=1
    print('공백', count,'개')

In 'userId' column
na 0 개 공백 25 개
na 0 개 공백 2 개
na 7120 개 공백 0 개
na 934 개 공백 0 개
na 0 개 공백 4 개
na 2730 개 공백 0 개


In [8]:
##For user-based CF
for csv in list_csv:
    df_raw = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='ms949')
    df_raw = df_raw.drop(df_raw.columns[3:],axis=1)
#replace space to NaN when cell only have space
    df_raw['placeId'] = df_raw['placeId'].replace('^ *$', float('nan'), regex=True)
    df_raw['userId'] = df_raw['userId'].replace('^ *$', float('nan'), regex=True)
    df_raw['score'] = df_raw['score'].replace('^ *$', float('nan'), regex=True)
#drop rows with NaN in placeId, userId, score
    before = len(df_raw)
    df_raw = df_raw.dropna(subset=['placeId','userId','score'])
    after = len(df_raw)
    print(before - after, 'entries droped in ', csv)
    df_raw = df_raw.sort_values(by=['userId']) 
#rescale score as 1~5
    df_raw['score'] = df_raw['score']/10
    df_raw['score'] = df_raw['score'].apply(int)
#unicode replacement
    for i in df_raw['userId']:
        i = re.sub(r'\\u[0-9a-f]{4}', unicode, i)
    df_raw['userId'] = df_raw['userId'].apply(lambda x:
    re.sub(r'\\u[0-9a-f]{4}', unicode, x))
#reset index and save
    df = df_raw.reset_index(drop=True)
    df.to_csv(csv+'_user.csv',mode='w',index=False)

25 entries droped in  data/kor/attraction_review
2 entries droped in  data/kor/hotel_review
7120 entries droped in  data/kor/restaurant_review
934 entries droped in  data/eng/eng_attraction_review
4 entries droped in  data/eng/eng_hotel_review
2730 entries droped in  data/eng/eng_restaurant_review


In [9]:
#0,0,25,15,8,37
##For tag-based CF
for csv in list_csv:
    df_raw = pd.read_csv(csv+'.csv',
                         names=['placeId','userId','score','title','review'],
                         encoding='ms949')
    df_raw = df_raw.drop(columns=['userId'])
#replace NaN in title and review to blank string
    df_raw['title'].fillna('', inplace=True)
    df_raw['review'].fillna('', inplace=True)
#merge title into review
    df_raw['review'] = df_raw.title + ' ' + df_raw['review']
    df_raw = df_raw.drop(columns=['title'])
#replace space to NaN when cell only have space
    df_raw['placeId'] = df_raw['placeId'].replace('^ *$', float('nan'), regex=True)
    df_raw['review'] = df_raw['review'].replace('^ *$', float('nan'), regex=True)
    df_raw['score'] = df_raw['score'].replace('^ *$', float('nan'), regex=True)
#drop rows with NaN in placeId, review, score
    before = len(df_raw)
    df_raw = df_raw.dropna(subset=['placeId','review','score'])
    after = len(df_raw)
    print(before - after, 'entries droped in ', csv)
    df_raw = df_raw.sort_values(by=['placeId']) 
#replace html tags and misinterpreted characters
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(r'<br/>|\\n',' ',x))
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(r'&#39;','\'',x))
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(r'&quot;','\'',x))
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(r'&amp;',' and ',x))
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(r'&gt;','',x))
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(r'&lt;','',x))
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(r'\';','\'',x))
#replace multiple spaces into one
    df_raw['review'] = df_raw['review'].apply(lambda x: re.sub(' +',' ',x))
#rescale score as 1~5
    df_raw['score'] = df_raw['score']/10
    df_raw['score'] = df_raw['score'].apply(int)
#unicode replacement
    for i in df_raw['review']:
        i = re.sub(r'\\u[0-9a-f]{4}', unicode, i)
    df_raw['review'] = df_raw['review'].apply(lambda x:
    re.sub(r'\\u[0-9a-f]{4}', unicode, x))
#reset index and save
    df_raw = df_raw.reset_index(drop=True)
    df_raw.to_csv(csv+'_tag.csv',mode='w')

0 entries droped in  data/kor/attraction_review
0 entries droped in  data/kor/hotel_review
25 entries droped in  data/kor/restaurant_review
15 entries droped in  data/eng/eng_attraction_review
8 entries droped in  data/eng/eng_hotel_review
37 entries droped in  data/eng/eng_restaurant_review
